In [11]:
from readrnx import readrnxobs, readrnxnav
from functions import satpos
from functions import hirvonen
import numpy as np
import math
import math as mat
# path to the broadcast navigation data - navigation RINEX file
nav_file = "BRDC00WRD_R_20213180000_01D_GN.rnx"
# path to the observation data - observation RINEX file
obs_file = "KRA100POL_R_20213180000_01D_30S_MO.rnx"
  
# define time of observation file - should be the time the same that in the observation file
# for first epoch only:
time_start = [2021, 11, 14,  8,  30,  0]
time_end = [2021, 11, 14,  8,  30,  0]

# reading the observation data from the file
obs,iobs2 = readrnxobs(obs_file, time_start, time_end, 'G')

# reading the navigation data from the file:
nav, inav = readrnxnav(nav_file)
i2 = iobs2[:,0]!=0
iobs = iobs2[i2,:]
# defining the approximate receiver coordinates: X,Y,Z
Xtrue = np.array([3856937.5695, 1397749.6854, 4867719.1135 ])

# some settings:
el_mask = 10 # elevation mask/cut off in degrees

# for the calculations only of the first epoch in the file, t is equal to 0 (because the data are from Sunday),
# week equal to 2184. If uou want to make calculations for different epochs, you can use the function date2tow(data), which can
# be found in the readrnx file, using the same as in the project 1

week = 2184
tr = 8*3600 + 30*60

# taking only the observations from the specified epoch
i = iobs[:,2]==tr
iobst = iobs[i,:]

# observation from one epoch - Pseudorange obs

#PRNs of satellites from the epoch
sats = iobst[:,0]

# create a vector of ro (geometric distance) for all satellites, in the first epoch ro can be tau*C, 
#where tau is approximate transmission time, equal to 0.07s
# in the first epoch tau should be 0.07 and ro should be tau*C
wE = 7.2921151467e-5
C = 299792458.0


alfa = [1.2107e-08, -7.4506e-09, -1.1921e-07, 5.96053e-08]
beta = [9.8304e+04, -8.1920e+04, -1.9661e+05, 4.5875e+05]

#tgps = 0
#el = 30
#az = 180
#fi = 50
#lam = 20

xr_pandas = []
dtr_pandas = []
dI_pandas = []
array_elevation = []


tr = 8*3600 + 30*60
def function():
    Xr0 = np.array([3850000.0000, 1390000.0000, 4860000.0000])    
    dtr = 0
    tau = 0.07
    ro_1 = np.ones((len(sats),1))*tau*C
    for i in range(5):
        n = 1
        L_new = []
        ts_new = []
        Xsr_new =[]
        ro_new = []
        Pcalc_new = []
        A1_row_1 = []
        for isat, sat in enumerate(sats):
            
            tau = ro_1[isat]/C
            
            #print(tau)
        

            # calculate transmission epoch: tr = t - tau - dtr, dtr is the receiver clock error, in the first iteration, it is zero
            ts = tr - tau + dtr
            
            
            # taking the ephemerides for one satellite - it should be an array of nav data, use (inav==sat)
            nav1sat = nav[(inav==sat),:]
       

            # calculating the coordinates of satellites, for the transmission epoch, use the satpos function, see the input values in the code
            # the outputs should be the XYZ and dts: inputs: satpos(week, tr, nav1sat)
             #!!!!!!!!!!!!!!!!!!!!
            Xs, dts = satpos(week, ts, nav1sat)
            

           # rotation of the satellite coordinates due to rotation of the earth, use the rotation around Z-axis
            X_rot = np.array([[math.cos(wE*tau),math.sin(wE*tau), 0],
                             [-math.sin(wE*tau),math.cos(wE*tau), 0],
                             [0,0,1]])
        

            Xs_rot = np.dot(X_rot,Xs)
        
        
          

            B,L,h = hirvonen(Xtrue[0],Xtrue[1],Xtrue[2])

              # satellite-receiver vector
            Xsr = Xs_rot - Xr0
            

            # the geometric distance between the satellite and the receiver
            ro = math.sqrt((Xsr[0]**2) + (Xsr[1]**2) + (Xsr[2]**2))
            ro_1[isat] = ro 
          
            # elevation and azimuth calculations
            Rneu = np.array([[-np.sin(B)*np.cos(L), -np.sin(L) , np.cos(B)*np.cos(L) ],
                     [-np.sin(B)*np.sin(L),  np.cos(L) , np.cos(B)*np.sin(L) ],
                     [ np.cos(B), 0 , np.sin(B)]])
            Rneu_Transpose = Rneu.T
            Xsrneu = np.matmul(Rneu_Transpose,Xsr)

            # if elevation above elevation mask, do the following calculations

            azimuth = np.arctan2(Xsrneu[1],Xsrneu[0])
            array_azimuth_angle = np.rad2deg(azimuth)

            numerator = Xsrneu[2]
            denominator = np.sqrt((Xsrneu[0]**2)+(Xsrneu[1]**2)+(Xsrneu[2]**2))
            elevation_angle = np.arcsin(numerator/denominator)
            array_elevation_angle = np.rad2deg(elevation_angle)
        

            if array_elevation_angle>10:
                

    
                els = array_elevation_angle/180
                azs = array_azimuth_angle/180
                fis = B/np.pi
                lams = L/np.pi

                #1.  Earth-centered angle of Ionospheric Pierce Point (IPP)
                psi = 0.0137/(els+0.11) - 0.022

                #2. Latitude of Ionospheric Pierce Point (IPP)
                fi_ipp =fis + psi * np.cos(np.deg2rad(array_azimuth_angle))

                if fi_ipp>0.416:
                    fi_ipp = 0.416
                elif fi_ipp<-0.416:
                    fi_ipp = -0.416

                #3. Logitude of Ionospheric Pierce Point (IPP)
                lam_ipp = lams + (psi * np.sin(np.deg2rad(array_azimuth_angle))/np.cos(fi_ipp*np.pi))

                #4. Geomagmetic latitude of Ionospheric Pierce Point (IPP)
                fi_mag_ipp = fi_ipp + 0.064 * np.cos((lam_ipp - 1.617)*np.pi)

                #5. local time
                tshow = 43200 * lam_ipp + tr

                t = math.fmod(tshow,86400) 


                if t > 86400:
                    t = t-86400
                elif t<0:
                    t = t +86400

                #6. Amplitude f ionospheric delay  
                aion = alfa[0] + alfa[1]*fi_mag_ipp + alfa[2]*fi_mag_ipp**2 + alfa[3]*fi_mag_ipp**3

                if aion<0:
                    aion = 0

                #7. Period of Ionospheric Delay

                pion = beta[0] + beta[1]*fi_mag_ipp + beta[2]*fi_mag_ipp**2 + beta[3]*fi_mag_ipp**3

                if pion<72000:
                    pion = 72000

                #8. Phase of iono delay
                fi_ion = 2*np.pi * (t-50400)/pion

                #9. Slant factor 
                mf = 1+16*(0.53-els)**3



                #10. Ionospheric delay
          
                

                if abs(fi_ion) <= np.pi/2:
                    dI = C * mf *(5 * 10**-9 + aion*(1-(fi_ion**2)/2 + (fi_ion**4)/24))
                  

                elif abs(fi_ion)>np.pi/2:
                    dI = C *mf *5 * 10**-9
                    
        
                
                # Trospheric delay
                p0 = 1013.25
                t0=291.5
                Rh0=0.5
                # because from the function hirvone has diffent from geoid and ellipsoid is 40 meters
                p=p0*(1-(0.0000226*(h-40)))**5.225
                t=t0-0.0065*(h-40)
                Rh=Rh0*mat.exp(-0.0006396*(h-40))
                e=6.11*Rh*10**(7.5*(t-273.15)/(t-35.85))
                Tdz=0.002277*p
                twz=0.002277*(1255/(t+0.05))*e   
                trop=1/(np.sin(elevation_angle))*(Tdz+twz)
               
        
                Pcalc = ro + C*dtr-C*dts + trop +dI
        
                #Pcalc_new.append(Pcalc)

                L = Pcalc - obs[isat]
                L_new.append(L)
                ts_new.append(ts)
                Xsr_new.append(Xsr)
                ro_new.append(ro)
                ro_copy = ro
                tau = ro_copy/C
                
                
        dI_pandas.append(dI)
              
            
        for j in range(len(L_new)): 
            A1_row = np.array([[(-Xsr_new[j][0])/ro_new[j]],[(-Xsr_new[j][1])/ro_new[j]],[(-Xsr_new[j][2])/ro_new[j]],[1]])
            A1_row_1.append(A1_row)


        A1 = np.array(A1_row_1)
        A1 = A1.squeeze(2)
        A1_transpose = np.array(A1).T
        x = -np.linalg.inv(A1_transpose@A1)@A1_transpose@L_new
        x = x.squeeze()
        xr = Xr0[0]+x[0]
        yr = Xr0[1]+x[1]
        zr = Xr0[2]+x[2]

        Xr = np.array([xr,yr,zr])
        xr_pandas.append(Xr)
        dtr_new = np.array(dtr+x[-1]/C)   
        dtr_pandas.append(dtr_new)
        Xr0 = Xr
        dtr = dtr_new

    return( xr_pandas, dtr_pandas)
            
df = function()
df

([array([3856933.98696762, 1397756.19132946, 4867711.346157  ]),
  array([3856936.08842946, 1397751.16835848, 4867716.98676777]),
  array([3856936.07115194, 1397751.17907715, 4867716.97296068]),
  array([3856936.07115328, 1397751.17908098, 4867716.97296236]),
  array([3856936.07115329, 1397751.17908097, 4867716.97296236])],
 [array(-1.72990648e-08),
  array(-1.27437201e-08),
  array(-1.28000563e-08),
  array(-1.2800044e-08),
  array(-1.2800044e-08)])

In [9]:
# Stastical in pandas
import pandas as pd
import matplotlib.pyplot as plt

receiver_position = np.array(df[0]).T
receiver_x = receiver_position[0]
receiver_y = receiver_position[1]
receiver_z = receiver_position[2]
dtr = df[1]


data_all = np.concatenate((receiver_x,receiver_y,receiver_z,dtr))
data_all = data_all.reshape(4,5)
data_all = data_all .T


print('Receiver refereence corrdiante with the calcualted reciver coordinate Without trospoheric and Ino correction')
data_pandas = pd.DataFrame(data_all , columns =['x','y','z','dtr'])
data_pandas 


Receiver refereence corrdiante with the calcualted reciver coordinate Without trospoheric and Ino correction


,x,y,z,dtr
0,3.856934e+06,1.397756e+06,4.867711e+06,-1.729906e-08
1,3.856936e+06,1.397751e+06,4.867717e+06,-1.274372e-08
2,3.856936e+06,1.397751e+06,4.867717e+06,-1.280006e-08
3,3.856936e+06,1.397751e+06,4.867717e+06,-1.280004e-08
4,3.856936e+06,1.397751e+06,4.867717e+06,-1.280004e-08
